# Inversión Fija Bruta
INEGI

In [1]:
# library 
Packages <- c("tidyverse","lubridate","ggplot2", "dplyr", "seasonal", "ggseas", "ggfortify", "forecast", "mFilter")
invisible(lapply(Packages, library, character.only=TRUE))

#set strings as factors to false 
options(stringsAsFactors=FALSE)


Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.2.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date


Attaching package: 'seasonal'

The following object is masked from 'package:tibble':

    view

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Registered S3 methods overwritten by 'forecast':
  method                 from     
  autoplot.Arima         ggfortify
  autoplot.acf   

In [28]:
mytimeseries <- function(var, lab_var){
     # raw_data 
    rawdata <- read_csv("data/ifb.csv")
    rawdata <- data.frame(mutate(rawdata, time = make_date(year, mes) ))
    #head(rawdata,3)

    #selección de datos, 2010 en adelante
    wrk <- rawdata %>% 
        filter(time>=as.Date("2010-01-01"))
    wrk <- ts(as_tibble(wrk),frequency=12, start=c(2010,1))
    
    raw_data <- wrk[,var]
    # Seasonal adjustment X13
    seas_adj <- seas(raw_data)
    #str(seas_adj$data)
    #summary(seas_adj$data)
    #head(seas_adj$data,3)
    seas_adj$model$arima
    arima <- seas_adj$model$arima$model
    modelo <- paste(" Modelo ARIMA:",arima,sep="")
    
    # HP filter 
    hp_data <- hpfilter(seas_adj$data[,"seasonaladj"])
    #summary(hp_data$trend)
    #str(hp_data$trend)

    #plot_data

    plot_data <- ts(
        mutate(as_tibble(seas_adj$data), 
               hp_trend = hp_data$trend, 
               rawdata = raw_data, 
               year = wrk[,"year"], 
               mes=wrk[,"mes"], 
               time = make_date(year,mes)
              ), frequency=12, start=c(2010,1), end=c(2019,8)
                    )
    
    csv_data <- subset(data.frame(plot_data), select=c("year","mes","rawdata","seasonaladj","trend","hp_trend"))
    
    csvpath <- paste("output/",var,sep="")
    csvpath <- paste(csvpath,".csv", sep="")
    write_csv(csv_data, path=csvpath)

    # graph   
    sub_lab <- "Índice base 2013 = 100"
    
    autoplot(plot_data[,c("seasonaladj", "trend", "hp_trend")], size=.8) + 
        labs( title = lab_var, 
             subtitle = sub_lab, 
             x = "", 
             y = "", 
             caption=paste("Fuente: Elaboración Secretaría de Economía con datos de INEGI.",modelo,sep="")
        ) + 
        theme_bw() + 
        theme(
            plot.title = element_text(hjust=0.5),
            plot.subtitle = element_text(hjust=0.5),
            legend.position="bottom", 
            plot.caption = element_text(hjust=0)
        ) + 
        scale_colour_manual(
            values=c("blue","red", "black"), 
            name="", 
            labels=c("Serie desestacionalizada", "Tendencia-ciclo", "Filtro Hodrick–Prescott")
        ) 

    figname <- paste("fig/",var,sep="")
    figname <- paste(figname,".png", sep="")

    ggsave(figname, width=18,height=12,units="cm")
    rm(rawdata, wrk, raw_data, seas_adj, arima, modelo, hp_data, plot_data, csv_data,sub_lab, figname)

    return(var)
}



In [29]:
# generación de series 
mytimeseries("ifb", "Inversión Fija Bruta")
mytimeseries("ifb_c", "IFB, Construcción")
mytimeseries("ifb_c_res", "IFB, Construcción Residencial")
mytimeseries("ifb_c_nr", "IFB, Construcción No Residencial")
mytimeseries("ifb_mq", "IFB, Maquinaria y Equipo")
mytimeseries("ifb_mq_nal","IFB, Maquinaria y Equipo: Nacional" )
mytimeseries("ifb_mq_nal_tran", "IFB, Maquinaria y Equipo: Nacional, Eq.Transporte")
mytimeseries("ifb_mq_nal_mq", "IFB, Maquinaria y Equipo: Nacional, Maq. Eq.")
mytimeseries("ifb_mq_im", "IFB, Maquinaria y Equipo: Importado")
mytimeseries("ifb_mq_im_tran", "IFB, Maquinaria y Equipo: Importado, Eq.Transporte")
mytimeseries("ifb_mq_im_mq", "IFB, Maquinaria y Equipo: Nacional, Maq. Eq.")


Parsed with column specification:
cols(
  .default = col_double()
)
See spec(...) for full column specifications.


[1] "ifb"

Parsed with column specification:
cols(
  .default = col_double()
)
See spec(...) for full column specifications.


[1] "ifb_c"

Parsed with column specification:
cols(
  .default = col_double()
)
See spec(...) for full column specifications.


...

[1] "ifb_mq_im"

Parsed with column specification:
cols(
  .default = col_double()
)
See spec(...) for full column specifications.


[1] "ifb_mq_im_tran"

Parsed with column specification:
cols(
  .default = col_double()
)
See spec(...) for full column specifications.


[1] "ifb_mq_im_mq"

## Inversión Fija Bruta al mes de agosto de 2019 

![](fig/ifb.png)

![](fig/ifb_c.png)

![](fig/ifb_c_res.png)

![](fig/ifb_c_nr.png)

![](fig/ifb_mq.png)

![](fig/ifb_mq_nal.png)

![](fig/ifb_mq_nal_mq.png)

![](fig/ifb_mq_nal_tran.png)

![](fig/ifb_mq_im.png)

![](fig/ifb_mq_im_mq.png)
